In [1]:
def sortedTime(database): 
    for user in database:
        data = database.get(user)
        for date in data:
            database[user][date].sort()
    
    return database

In [2]:
def createDatabase():
    f = open("userlog.log")
    lines = f.readlines()
    database = {}
    for line in lines:
        arr = line.split()
        user = str(arr[4])
        date = str(arr[0])
        if user in database:
            if date in database[user]: 
                database[user][date].append([str(arr[1]),str(arr[2]),str(arr[3])])
            else: 
                database[user][date] = []
                database[user][date].append([str(arr[1]),str(arr[2]),str(arr[3])])
        else:
            database[user] = {}
            database[user][date] = []
            database[user][date].append([str(arr[1]),str(arr[2]),str(arr[3])])
    return database

In [3]:
def sortDates(database):
    for user in database:
        temp = sorted(database[user][1])
        database[user] = [database[user][0],temp]
    return database

In [4]:
def sus_activity():
    database = sortedTime(createDatabase())
    suspicious = {}
    counter = 0
    for user in database:
        for date in database[user]:
            arr = database[user][date]
            logins = len([x for x in arr if x[1]=="login"])
            sus_logins = len([x for x in arr if ((int(x[0].split(":")[0]))>= 0 or int(x[0].split(":")[0])<=5) and (x[1]=="login")])
            if logins > 5 or sus_logins > 0:
                counter += 1
                if user in suspicious:
                    data = suspicious[user]
                    num = data[0]
                    lst = data[1]
                    lst.append(date)
                    suspicious[user] = [num+1,lst]
                else:
                    suspicious[user] = [1,[date]]
                    
    #write to file 
    sorted_database = sortDates(suspicious)
    t = open("suspicious_report.txt","w")
    t.write("    =======================\n")
    t.write("=== Suspicious Activity "+"("+str(counter)+" cases)"+" ===\n")
    t.write("    ========================\n")
    for user in sorted_database:
        num_user = sorted_database[user][0]
        t.write(user+" "+str(num_user)+"\n")
        for date in sorted_database[user][1]:
            t.write("\t"+"DATE: "+"["+date+"]"+" ---\n")
            for data in database[user][date]:
                t.write("\t\t"+data[0]+"  "+data[1]+"  "+data[2]+"\n")
            t.write("\n")
            
    t.close()

sus_activity()
#print(open("suspicious_report.txt","r").read())

In [5]:
def irresponsible_glitches():
    # Open log
    f = open("userlog.log")
    # Make it readable
    lines = f.readlines()
    # Create database
    database = {} 
    # Iterate through file
    for line in lines:
        
        # Make the current string a list
        arr = line.split()
                
        # If the email is already in server, check if the date is
        if str(arr[4]) in database:
            
            # If date is in database, simply increment/decrement counter
            if str(arr[0]) in database[str(arr[4])]:
                
                if str(arr[2]) == "login":
                    database[str(arr[4])][str(arr[0])] += 1
                elif str(arr[2]) == "logout":
                    database[str(arr[4])][str(arr[0])] -= 1
                    
            
            # If date not in database, make the date, activity pair        
            elif str(arr[2]) == 'login':
                database[str(arr[4])][str(arr[0])] = 1
            else:
                database[str(arr[4])][str(arr[0])] = -1
             
            
        # If the email isn't in the server, add it + check the activity done
        # No need to check any date because if the email isnt in the dict, the date won't be either
        else:
            
            # Create email -> k,v pair
            database[str(arr[4])] = {}
            
            # Check line instance of activity and create the activity counter
            if str(arr[2]) == 'login':
                database[str(arr[4])][str(arr[0])] = 1
            else:
                database[str(arr[4])][str(arr[0])] = -1
                
    return database

In [6]:
def write_irres():
    other_database = irresponsible_glitches()
    database = sortedTime(createDatabase())

    n_cases = 0
    for email in other_database:
        for date in other_database[email]:
            if other_database[email][date] > 0:
                n_cases += 1
                
    finale = {}
    for email in other_database:
        finale[email] = {}
        for date in other_database[email]:
                
            if other_database[email][date] > 0:

                finale[email][date] = database[email][date]
   
    t = open("irresponsible_report.txt","w")
    t.write("    =======================\n")
    t.write("=== Irresponsible Activity "+"("+str(n_cases)+" cases)"+" ===\n")
    t.write("    ========================\n")
     
        
    for email in finale:
        
        t.write(str(email) +" "+ str(len(finale[email])) + "\n")
        
        for date in finale[email]:
            t.write("\t"+"DATE: "+"["+str(date)+"]"+" ---\n")
            for data in finale[email][date]:
                t.write("\t\t"+data[0]+"  "+data[1]+"  "+data[2]+"\n")
                t.write("\n")
  
    t.close()
    
    
write_irres()
#print(open("irresponsible_report.txt","r").read())

In [7]:
def write_glitch():
    other_database = irresponsible_glitches()
    database = sortedTime(createDatabase())

    n_cases = 0
    for email in other_database:
        for date in other_database[email]:
            if other_database[email][date] < 0:
                n_cases += 1
                
    finale = {}
    for email in other_database:
        finale[email] = {}
        for date in other_database[email]:
                
            if other_database[email][date] < 0:

                finale[email][date] = database[email][date]
   
    t = open("glitch_report.txt","w")
    t.write("    =======================\n")
    t.write("=== Glitch Activity "+"("+str(n_cases)+" cases)"+" ===\n")
    t.write("    ========================\n")
     
        
    for email in finale:
        
        t.write(str(email) +" "+ str(len(finale[email])) + "\n")
        
        for date in finale[email]:
            t.write("\t"+"DATE: "+"["+str(date)+"]"+" ---\n")
            for data in finale[email][date]:
                t.write("\t\t"+data[0]+"  "+data[1]+"  "+data[2]+"\n")
                t.write("\n")
  
    t.close()
    
    
write_glitch()
#print(open("glitch_report.txt","r").read())

In [8]:
def domain_count():
    database = sortedTime(createDatabase())
    file_name = "domain_report.txt"
    domain_database = {}
    for user in database:
        domain_str = user.split("@")
        if str(domain_str[1]) in domain_database:
            domain_database[str(domain_str[1])] += 1
        else:
            domain_database[str(domain_str[1])] = 1
    f = open(file_name, "w")
    f.write("    =======================\n")
    f.write("=== Domain Count " + "("+str(len(domain_database)) +" domains)"+ " ===\n")
    f.write("    =======================\n")
    for domain,count in domain_database.items():
        f.write(domain+":"+"  "+str(count)+"\n")
    f.close() 
    
domain_count()
#print(open("domain_report.txt","r").read())